In [1]:
# importing labraries
import numpy as np
from scipy.stats import norm
import pandas as pd
import math

> # PART I

In [2]:
def discount_T(T, r):
    return np.exp(-r * (T))

> ## `Black-Scholes model` 

## $ V_c = S_0Φ(d_1) − Ke^{−rT} Φ(d_2) $

In [3]:
# Lognormal model
def Black_Scholes(S, K, sigma, T, r, cash=1):
    d1 = (math.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    decimal_places = 6
    
    # Define the to be returned DataFrame data as NaN
    data = np.NaN
    # Define your row names and column names
    row_names = ['Van', 'DCN', 'DAN']
    col_names = ['Call', 'Put']

    # Create the DataFrame
    BS_value = pd.DataFrame(data, index=row_names, columns=col_names)
    
    # Value on Vanilla call/put 
    call_value = (S * norm.cdf(d1) - K * discount_T(T, r) * norm.cdf(d2))
    BS_value.loc['Van', 'Call'] = round(call_value, decimal_places)        
    put_value = (-S * norm.cdf(-d1) + K * discount_T(T, r) * norm.cdf(-d2))
    BS_value.loc['Van', 'Put'] = round(put_value, decimal_places)
    
    # Value on Digital Cash-or-Nothing call/put     
    call_value = discount_T(T, r) *\
        (cash * norm.cdf(d2))
    BS_value.loc['DCN', 'Call'] = round(call_value, decimal_places)        
    put_value = discount_T(T, r) *\
        (cash * norm.cdf(-d2))
    BS_value.loc['DCN', 'Put'] = round(put_value, decimal_places)    
    
    # Value on Digital Asset-or-Nothing call/put     
    call_value = \
        (S * norm.cdf(d1))
    BS_value.loc['DAN', 'Call'] = round(call_value, decimal_places)        
    put_value = \
        (S * norm.cdf(-d1))
    BS_value.loc['DAN', 'Put'] = round(put_value, decimal_places) 
    
    return BS_value

In [4]:
(S, K, Sigma, T, r, amt) =\
(1, 1, 0.15, 0.5, 0.01, 3)
Black_Scholes(S, K, Sigma, T, r, amt)

,Call,Put
Van,0.044730,0.039742
DCN,1.485502,1.499536
DAN,0.539897,0.460103


> ## `Bachelier model`

In [5]:
# Normal model
def Bachelier(S, K, sigma, T, r, cash = 1):
    d = (K - S) / (sigma * np.sqrt(T))         
    decimal_places = 6
    # Define the to be returned DataFrame data as NaN
    data = np.NaN
    # Define your row names and column names
    row_names = ['Van', 'DCN', 'DAN']
    col_names = ['Call', 'Put']

    # Create the DataFrame
    Bach_value = pd.DataFrame(data, index=row_names, columns=col_names)
    
    # Value on Vanilla call/put 
    call_value = discount_T(T, r) *\
        ((S - K) * norm.cdf(-d) + sigma * math.sqrt(T) * norm.pdf(-d))
    Bach_value.loc['Van', 'Call'] = round(call_value, decimal_places)        
    put_value = discount_T(T, r) *\
        ((K - S) * norm.cdf(d) + sigma * math.sqrt(T) * norm.pdf(d))
    Bach_value.loc['Van', 'Put'] = round(put_value, decimal_places)
    
    # Value on Digital Cash-or-Nothing call/put     
    call_value = discount_T(T, r) *\
        (cash * norm.cdf(-d))
    Bach_value.loc['DCN', 'Call'] = round(call_value, decimal_places)        
    put_value = discount_T(T, r) *\
        (cash * norm.cdf(d))
    Bach_value.loc['DCN', 'Put'] = round(put_value, decimal_places)    
    
    # Value on Digital Asset-or-Nothing call/put     
    call_value = discount_T(T, r) *\
        ((S) * norm.cdf(-d) + sigma * math.sqrt(T) * norm.pdf(-d))
    Bach_value.loc['DAN', 'Call'] = round(call_value, decimal_places)        
    put_value = discount_T(T, r) *\
        ((S) * norm.cdf(d) - sigma * math.sqrt(T) * norm.pdf(d))
    Bach_value.loc['DAN', 'Put'] = round(put_value, decimal_places)      
    
    return Bach_value

In [6]:
(S, K, Sigma, T, r, amt) =\
(1.05, 1, 0.15, 0.5, 0.01, 10)
Bachelier(S, K, Sigma, T, r, amt)

,Call,Put
Van,0.071572,0.021821
DCN,6.779259,3.170865
DAN,0.749498,0.295265


> ## `Black76 model`

In [7]:
def Black76(F, K, sigma, T, r, cash=1):
    S = F * discount_T(T, r)
    B76 = Black_Scholes(S, K, sigma, T, r, cash)
    return B76

In [8]:
(F, K, Sigma, T, r) =\
(127.09, 110, 0.08, 1, 0.1)
Black76(F, K, Sigma, T, r)

,Call,Put
Van,15.584193,0.120521
DCN,0.869760,0.035077
DAN,111.257848,3.737940


> ## `Displaced-diffusion model`

In [14]:
def Displaced_diffusion(F, K, sigma, T, r, cash=1, beta=0.7):
    DF = Black76(F/beta, K+F*(1-beta)/beta, sigma*beta, T, r, cash)
    return DF

In [15]:
(F, K, Sigma, T, r, cash, beta) =\
(100, 105, 0.2, 1, 0.05, 100, 0.3)
Displaced_diffusion(F, K, Sigma, T, r, cash, beta)

,Call,Put
Van,5.501525,10.257673
DCN,37.140856,57.982086
DAN,131.161422,185.915053


In [16]:
(
Black_Scholes(S=100, K=105, sigma=0.2, T=1, r=0.05, cash = 100),
Bachelier(S=100, K=105, sigma=0.2, T=1, r=0.05, cash = 100),
Black76(F=112, K=105, sigma=0.2, T=1, r=0.05, cash=100),
Displaced_diffusion(F=100, K=105, sigma=0.2, T=1, r=0.05, cash=100, beta=0.2)
)

(          Call        Put
 Van   8.021352   7.900442
 DCN  44.001411  51.121532
 DAN  54.222833  45.777167,
      Call        Put
 Van   0.0   4.756147
 DCN   0.0  95.122942
 DAN   0.0  95.122942,
           Call        Put
 Van  11.973201   5.314595
 DCN  55.943003  39.179939
 DAN  70.713354  35.824342,
            Call        Put
 Van    5.483823   10.23997
 DCN   37.483943   57.63900
 DAN  194.777732  280.83698)

># Part I Combined function

In [17]:
def Option_value(S, K, sigma, T, r, func, cash=100, beta=0.7):
    """
    Calculate the Black-Scholes option price for a European call or put.

    Parameters in list_val in sequence of :
    [S, K, T, r, sigma, cash, d]

    - S(F): 
        S-> Current stock price in Black_Scholes and Bachelier model
        OR
        F-> Future price in Black 76 model and displaced-diffusion
    - K: Strike price
    - T: Time to maturity (in years)
    - r: Risk-free interest rate (annualized)
    - sigma: Volatility (annualized)
    - cash: for Digital Cash-Or-Nothing option
    - beta: Displacement scale of the underlying asset price.
        for Displaced-diffusion model only
    
    - func: choice of models as below
        - Black_Scholes
        - Bachelier
        - Black76
        - Displaced_diffusion
    
    Returns:
    - Option price as DataFrame
    """    
    if func == Displaced_diffusion:
        result = func(S, K, sigma, T, r, cash, beta)
    else:
        result = func(S, K, sigma, T, r, cash)
    return result

In [18]:
## func = Black_Scholes / Bachelier / Black76 / Displaced_diffusion
(S, K, sigma, T, r, cash, beta) = \
(100, 105, 0.2, 1, 0.1, 100, 0.3)
Option_value(S, K, sigma, T, r, Displaced_diffusion, cash, beta)

,Call,Put
Van,5.233213,9.757400
DCN,35.329475,55.154267
DAN,124.764604,176.847868
